## Import Libraries & Load Dataset

In [1]:
import os
import numpy as np
import pandas as pd
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, Dense, Flatten, Dropout, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint
import matplotlib.pyplot as plt

## Age Estimation Dataset

In [2]:
images , ages = [],[]
for path in os.listdir("/content/UTKFace"):
    img = cv2.imread(f"/content/UTKFace/{path}")
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    img = cv2.resize(img,(150,150))
    age = int(path.split("_")[0])
    images.append(img)
    ages.append(age)

In [3]:
images = pd.Series(images,name="images")
ages = pd.Series(ages,name="ages")
df = pd.concat([images,ages],axis=1)
df = df[df["ages"]<=80]
df1 = df[df["ages"]<=4]
df1 = df1.sample(frac=0.5)
df2 = df[df["ages"]>4]
df = pd.concat([df1,df2],axis=0)

In [4]:
df1 = df[(df["ages"]>=20) & (df["ages"]<=35)]
df1 = df1.sample(frac=0.3)
df2 = df[(df["ages"]<20) | (df["ages"]>35)]
df = pd.concat([df1,df2],axis=0)

In [5]:
X = np.array(df["images"].tolist())
Y = np.array(df["ages"].tolist())

In [6]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y,test_size=0.2,random_state=42)

In [8]:
image_generator = ImageDataGenerator(
    rescale = 1.0 / 255.0,
    horizontal_flip = True
)
image_generator_val = ImageDataGenerator(
    rescale = 1.0 / 255.0
)

In [ ]:
Train = image_generator.flow(
    X_train,
    Y_train,
    batch_size=32
)
Val = image_generator_val.flow(
    X_test,
    Y_test,
    batch_size=32
)

## Model API

In [10]:
input1 = Input(shape=(150,150,3))
conv1 = Conv2D(64,(3,3),activation="relu",padding="same")(input1)
maxpool1 = MaxPool2D(pool_size=(2,2))(conv1)
conv2 = Conv2D(128,(3,3),activation="relu",padding="same")(maxpool1)
maxpool2 = MaxPool2D(pool_size=(2,2))(conv2)
conv3 = Conv2D(256,(3,3),activation="relu",padding="same")(maxpool2)
maxpool3 = MaxPool2D(pool_size=(2,2))(conv3)
batchnormal = BatchNormalization()(maxpool3)
flat = Flatten()(batchnormal)
dense1 = Dense(256,activation="relu")(flat)
dense2 = Dense(128,activation="relu")(dense1)
drop = Dropout(0.5)(dense2)
dense3 = Dense(1,activation="linear")(drop)
model = Model(inputs=input1,outputs=dense3)

In [11]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 150, 150, 3)]     0         
                                                                 
 conv2d (Conv2D)             (None, 150, 150, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 75, 75, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 75, 75, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 37, 37, 128)      0         
 2D)                                                             
                                                                 
 conv2d_2 (Conv2D)           (None, 37, 37, 256)       295168

In [12]:
model.compile(optimizer="adam",loss="mean_absolute_error")
checkpoint = ModelCheckpoint("AgeEstimation.h5",monitor="val_loss",mode="min",save_best_only=True)

In [13]:
history = model.fit(Train,
                    epochs=20,
                    validation_data=Val,
                    callbacks=[checkpoint])

Epoch 1/20
369/369 [==============================] - 63s 162ms/step - loss: 14.2269 - val_loss: 25.1978
Epoch 2/20
369/369 [==============================] - 57s 155ms/step - loss: 11.2392 - val_loss: 11.3708
Epoch 3/20
369/369 [==============================] - 57s 155ms/step - loss: 10.1453 - val_loss: 8.8890
Epoch 4/20
369/369 [==============================] - 57s 155ms/step - loss: 9.4528 - val_loss: 8.0527
Epoch 5/20
369/369 [==============================] - 56s 152ms/step - loss: 9.0134 - val_loss: 8.1953
Epoch 6/20
369/369 [==============================] - 56s 152ms/step - loss: 8.5449 - val_loss: 8.2451
Epoch 7/20
369/369 [==============================] - 56s 152ms/step - loss: 8.4517 - val_loss: 8.2040
Epoch 8/20
369/369 [==============================] - 56s 152ms/step - loss: 8.0931 - val_loss: 8.2530
Epoch 9/20
369/369 [==============================] - 56s 152ms/step - loss: 7.8091 - val_loss: 13.5436
Epoch 10/20
369/369 [==============================] - 56s 152ms/st